## Do Not Run This Notebook!
This notebook is only there to document what was done during the data preprocessing.
In the absolute emergency of needing to run it again, please check file paths.
Some may need adding "../" before them.

In [1]:
import pandas as pd
import csv

In [1]:
%run import_data_

Continuing with existing version of data folder
Goodreads dataset loaded successfully as books_goodreads
Pandas dataframes (books_goodreads, books_big, book, users, ratings) loaded successfully
Columns in DataFrames 'users' and 'ratings' renamed
You can use the DataFrames 'books' or 'books_big' - they are exactly the same (big) dataset
loading books_ratings and books_users_ratings
Ready to go!


In [3]:
# input_file = 'data/big_detailed.csv'

In [4]:
# Define a function to process the CSV file
def process_csv(file_path):
    data = []
    
    # Open and read the CSV file
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        
        # Append rows with the third+ column as a list
        for row in reader:
            data.append([row[0], row[1], row[2:]])  # Always treat 3rd column as a list
    
    # Convert the list of rows to a pandas DataFrame
    df = pd.DataFrame(data, columns=['isbn', 'publish_date', 'subjects'])
    
    return df

# df = process_csv(input_file)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281472 entries, 0 to 281471
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   isbn          281472 non-null  object
 1   publish_date  281472 non-null  object
 2   subjects      281472 non-null  object
dtypes: object(3)
memory usage: 6.4+ MB


In [5]:
df.head()

,isbn,publish_date,subjects
0,isbn,publish_date,[genres]
1,0195153448,2003,"[Classical Mythology, Mythology, Classical, My..."
2,0002005018,2001,"[Actresses, Fiction, Literature, Sisters, Wome..."
3,0060973129,Error fetching data,"[Campaigns, History, History, Military, Milita..."
4,0374157065,1999,"[Influenza, History, Grippe humaine, Disease O..."


In [6]:
# The first row is the header with the wrong number of columns
df = df.iloc[1:]  # Slice the DataFrame to exclude the first row
df = df.reset_index(drop=True)
df.head()

,isbn,publish_date,subjects
0,0195153448,2003,"[Classical Mythology, Mythology, Classical, My..."
1,0002005018,2001,"[Actresses, Fiction, Literature, Sisters, Wome..."
2,0060973129,Error fetching data,"[Campaigns, History, History, Military, Milita..."
3,0374157065,1999,"[Influenza, History, Grippe humaine, Disease O..."
4,0393045218,1999,"[Antiquities, Bronze age, Mummies, Prehistoric..."


In [9]:
df.isbn.duplicated().sum()

54183

#### Clearly, some of the isbn's now have more than 1 row, how can we choose only 1 without running the risk of choosing the one with more missings?

In [10]:
# Define the set of values to remove
values_to_remove = {'No description found', 'No genres found', '', pd.NA}

# Function to remove specific values from a list
def remove_values(lst, values):
    return [item for item in lst if item not in values]

# Apply the function to the 'subjects' column
df['subjects'] = df['subjects'].apply(lambda x: remove_values(x, values_to_remove))

In [11]:
# Calculate the length of each list in the 'subjects' column
df['subjects_length'] = df['subjects'].apply(len)
df.subjects_length.value_counts()

subjects_length
1    243314
0     35139
2      3019
Name: count, dtype: int64

In [12]:
# Function to check if a list contains duplicates
def has_duplicates(lst):
    return len(lst) != len(set(lst))

# Apply the function to the 'subjects' column
df['has_duplicates'] = df['subjects'].apply(has_duplicates)

In [13]:
df.has_duplicates.value_counts()

has_duplicates
False    281411
True         61
Name: count, dtype: int64

In [14]:
# Define a function to drop the last element of a list
def drop_last_element(lst):
    return lst[:-1]

# Apply the function only to rows where 'has_duplicates' is True
df.loc[df['has_duplicates'], 'subjects'] = df.loc[df['has_duplicates'], 'subjects'].apply(drop_last_element)


In [15]:
df['has_duplicates'] = df['subjects'].apply(has_duplicates)

In [16]:
df.has_duplicates.value_counts()

has_duplicates
False    281472
Name: count, dtype: int64

In [17]:
df['subjects_length'] = df['subjects'].apply(len)
df.subjects_length.value_counts()

subjects_length
1    243375
0     35139
2      2958
Name: count, dtype: int64

In [18]:
df.shape

(281472, 5)

In [19]:
df.isbn.nunique()

227289

In [20]:
df.isbn.duplicated().sum()

54183

In [21]:
# Convert lists in 'subjects' column to frozenset
df['subjects'] = df['subjects'].apply(frozenset)

In [22]:
df.duplicated().sum() # this won't work as long as the 'subjects' column is a list

50032

In [23]:
df = df.drop_duplicates()

In [24]:
df.shape

(231440, 5)

In [25]:
df.isbn.duplicated().sum()

4151

In [26]:
df.isbn.nunique()

227289

#### Summary so far:
* Duplicates within 'subjects' removed
* Values 'No description found' and 'No genres found' were removed
* Also empty cells and pd.NA were removed
* The column 'subjects' was initially imported as a list (since some rows in .csv contain 2 fields for 'subject')
* The list was converted to a frozenset to make it possible to check for duplicates across all columns without taking the order of the 2 fields of 'subjects' into consideration
* Now there are only 4151 rows which include books that were already in previous rows, but the values of the other columns in these rows are not entirely identical

In [27]:
df.head()

,isbn,publish_date,subjects,subjects_length,has_duplicates
0,0195153448,2003,"(Classical Mythology, Mythology, Classical, My...",1,False
1,0002005018,2001,"(Actresses, Fiction, Literature, Sisters, Wome...",1,False
2,0060973129,Error fetching data,"(Campaigns, History, History, Military, Milita...",1,False
3,0374157065,1999,"(Influenza, History, Grippe humaine, Disease O...",1,False
4,0393045218,1999,"(Antiquities, Bronze age, Mummies, Prehistoric...",1,False


In [28]:
df = df.drop(columns=['subjects_length', 'has_duplicates'])

In [29]:
df.head()

,isbn,publish_date,subjects
0,0195153448,2003,"(Classical Mythology, Mythology, Classical, My..."
1,0002005018,2001,"(Actresses, Fiction, Literature, Sisters, Wome..."
2,0060973129,Error fetching data,"(Campaigns, History, History, Military, Milita..."
3,0374157065,1999,"(Influenza, History, Grippe humaine, Disease O..."
4,0393045218,1999,"(Antiquities, Bronze age, Mummies, Prehistoric..."


In [30]:
df.duplicated().sum()
# No more books are entirely duplicated

0

In [31]:
df.duplicated(subset=['isbn', 'publish_date'], keep=False).sum()
# 5850 books have the same isbn and publish_date but different subjects

5850

In [32]:
df.duplicated(subset=['isbn', 'subjects'], keep=False).sum()
# 1886 books have the same isbn and subjects but different publish_date

1886

##### Let's begin to tackle the publish_date column:

In [33]:
# Replace specific error strings with NaN
import numpy as np
df['publish_date'] = df['publish_date'].replace(['Error fetching data', 'No date found'], np.nan)


In [34]:
df.publish_date.isnull().sum() # 4034 books have no publish_date

4034

In [39]:
!pip install python-dateutil

In [41]:
import re # regular expressions library
import dateutil.parser as dparser # robust date parsing library

# Function to extract year from various formats of publish_date
def extract_year(date_str):
    try:
        # Try parsing the date using dateutil.parser
        parsed_date = dparser.parse(date_str, fuzzy=True)
        # Return the year from the parsed date
        return parsed_date.year
    except (ValueError, TypeError, OverflowError):
        # If there's a problem with parsing, return None or NaN
        return pd.NA

# Apply the function to the publish_date column
df['year'] = df['publish_date'].apply(extract_year)


In [42]:
df.year.isnull().sum() # 4328 books have no year

4328

In [43]:
df.head()

,isbn,publish_date,subjects,year
0,0195153448,2003,"(Classical Mythology, Mythology, Classical, My...",2003
1,0002005018,2001,"(Actresses, Fiction, Literature, Sisters, Wome...",2001
2,0060973129,NaN,"(Campaigns, History, History, Military, Milita...",<NA>
3,0374157065,1999,"(Influenza, History, Grippe humaine, Disease O...",1999
4,0393045218,1999,"(Antiquities, Bronze age, Mummies, Prehistoric...",1999


In [44]:
df.shape

(231440, 4)

#### Merging the scraped data with the original dataset

In [45]:
# let's merge the available data:
df_test = pd.merge(books_big, df, on='isbn', how='inner')

In [46]:
df_test.head()

,isbn,book_title,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l,publish_date,subjects,year
0,0195153448,Classical Mythology,Mark P. O. Morford,2002-01-01,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2003,"(Classical Mythology, Mythology, Classical, My...",2003
1,0002005018,Clara Callan,Richard Bruce Wright,2001-01-01,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,2001,"(Actresses, Fiction, Literature, Sisters, Wome...",2001
2,0060973129,Decision in Normandy,Carlo D'Este,1991-01-01,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,NaN,"(Campaigns, History, History, Military, Milita...",<NA>
3,0060973129,Decision in Normandy,Carlo D'Este,1991-01-01,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,1991,"(Campaigns, History, History, Military, Milita...",1991
4,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999-01-01,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,1999,"(Influenza, History, Grippe humaine, Disease O...",1999


In [47]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231440 entries, 0 to 231439
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   isbn                 231440 non-null  object        
 1   book_title           231440 non-null  object        
 2   book_author          231438 non-null  object        
 3   year_of_publication  227428 non-null  datetime64[ns]
 4   publisher            231438 non-null  object        
 5   image_url_s          231440 non-null  object        
 6   image_url_m          231440 non-null  object        
 7   image_url_l          231437 non-null  object        
 8   publish_date         227406 non-null  object        
 9   subjects             231440 non-null  object        
 10  year                 227112 non-null  object        
dtypes: datetime64[ns](1), object(10)
memory usage: 19.4+ MB


In [48]:
df_test.year_of_publication.isnull().sum() # 4012 missing values

4012

In [49]:
# Let's try to use the scraped 'year' values as replacement for missing 'year_of_publication':
df_test['year_of_publication'] = df_test['year_of_publication'].fillna(df_test['year'])

In [50]:
df_test.year_of_publication.isnull().sum() # reduced missing values to 813

813

In [51]:
# Now we can drop the scraped date columns:
df_test = df_test.drop(columns=['year', 'publish_date'])

In [52]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231440 entries, 0 to 231439
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 231440 non-null  object
 1   book_title           231440 non-null  object
 2   book_author          231438 non-null  object
 3   year_of_publication  230627 non-null  object
 4   publisher            231438 non-null  object
 5   image_url_s          231440 non-null  object
 6   image_url_m          231440 non-null  object
 7   image_url_l          231437 non-null  object
 8   subjects             231440 non-null  object
dtypes: object(9)
memory usage: 15.9+ MB


In [53]:
# let's extract the year from 'year_of_publication'
df_test['year_of_publication'] = pd.to_datetime(df_test['year_of_publication'], errors='coerce')
df_test['year_of_publication'] = df_test['year_of_publication'].dt.year

In [54]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231440 entries, 0 to 231439
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 231440 non-null  object 
 1   book_title           231440 non-null  object 
 2   book_author          231438 non-null  object 
 3   year_of_publication  230627 non-null  float64
 4   publisher            231438 non-null  object 
 5   image_url_s          231440 non-null  object 
 6   image_url_m          231440 non-null  object 
 7   image_url_l          231437 non-null  object 
 8   subjects             231440 non-null  object 
dtypes: float64(1), object(8)
memory usage: 15.9+ MB


In [55]:
df_test.head()

,isbn,book_title,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l,subjects
0,0195153448,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,"(Classical Mythology, Mythology, Classical, My..."
1,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"(Actresses, Fiction, Literature, Sisters, Wome..."
2,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,"(Campaigns, History, History, Military, Milita..."
3,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,"(Campaigns, History, History, Military, Milita..."
4,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,"(Influenza, History, Grippe humaine, Disease O..."


In [56]:
df_test['year_of_publication'].describe()

count    230627.000000
mean       1993.575739
std           8.492597
min        1806.000000
25%        1989.000000
50%        1996.000000
75%        2000.000000
max        2050.000000
Name: year_of_publication, dtype: float64

In [60]:
df_test.year_of_publication[df_test['year_of_publication'] > 2024].value_counts()
# year_of_publication
# 2030.0    4
# 2050.0    2
# 2038.0    1
# 2026.0    1
# Name: count, dtype: int64

Series([], Name: count, dtype: int64)

In [61]:
# Let's replace the future dates (above 2024) with NaN
df_test[df_test['year_of_publication'] > 2024].year_of_publication = np.nan

In [62]:
df_test['year_of_publication'].describe()

count    230619.000000
mean       1993.574285
std           8.488991
min        1806.000000
25%        1989.000000
50%        1996.000000
75%        2000.000000
max        2021.000000
Name: year_of_publication, dtype: float64

In [63]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231440 entries, 0 to 231439
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 231440 non-null  object 
 1   book_title           231440 non-null  object 
 2   book_author          231438 non-null  object 
 3   year_of_publication  230619 non-null  float64
 4   publisher            231438 non-null  object 
 5   image_url_s          231440 non-null  object 
 6   image_url_m          231440 non-null  object 
 7   image_url_l          231437 non-null  object 
 8   subjects             231440 non-null  object 
dtypes: float64(1), object(8)
memory usage: 15.9+ MB


In [64]:
# df_test has around 930 overall duplicates, let's remove them:
df_test = df_test.drop_duplicates()
df_test.duplicated().sum()

0

In [65]:
# But how many isbn's are still duplicated?
df_test.isbn.duplicated().sum() # 3221

3221

In [66]:
df_test.duplicated(subset=['isbn', 'subjects', 'year_of_publication'], keep=False).sum() # 0

0

In [67]:
# Books with duplicates that are ONLY different in 'year_of_publication':
df_test.duplicated(subset=['isbn', 'subjects'], keep=False).sum() # 26

26

In [68]:
df_test.isbn.isna().sum() # 0
df_test.isbn.isnull().sum() # 0
# So far, no missing values in 'isbn'

0

##### Books with duplicates that are ONLY different in 'subjects'

In [69]:
# Books with duplicates that are ONLY different in 'subjects':
df_test.duplicated(subset=['isbn', 'year_of_publication'], keep=False).sum()
# 6404

### (I checked, and the other columns are the same for these duplicates)

6404

In [70]:
# Let's handle this by keeping only the rows with the most data available in 'subjects':

# Step 1: Create a new column to store the length of the 'subjects' frozenset
df_test['subjects_len'] = df_test['subjects'].apply(len)

# Step 2: Sort the dataframe by the relevant columns and 'subjects_len', keeping the longest 'frozenset' first
df_test_sorted = df_test.sort_values(by=['isbn', 'book_title', 'book_author', 'year_of_publication', 'publisher', 'image_url_s', 'image_url_m', 'image_url_l', 'subjects_len'], ascending=[True, True, True, True, True, True, True, True, False])
print('df_test_sorted number of missing isbn values:')
print(df_test_sorted.isbn.isna().sum()) 

# Step 3: Drop duplicates, keeping the row with the longest 'frozenset' (sorted first)
df_test_deduplicated = df_test_sorted.drop_duplicates(subset=['isbn', 'year_of_publication'], keep='first')
print('df_test_deduplicated number of missing isbn values:')
print(df_test_deduplicated.isbn.isna().sum()) 

# Step 4: Drop the 'subjects_len' column (no longer needed)
df_test_deduplicated = df_test_deduplicated.drop(columns='subjects_len')
print('df_test_deduplicated number of missing isbn values after finishing:')
print(df_test_deduplicated.isbn.isna().sum()) 

# Now df_test_deduplicated contains no duplicates, with the longest frozenset retained


df_test_sorted number of missing isbn values:
0
df_test_deduplicated number of missing isbn values:
0
df_test_deduplicated number of missing isbn values after finishing:
0


In [71]:
df_test_deduplicated.head()

,isbn,book_title,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l,subjects
219957,0001010565,Mog's Christmas,Judith Kerr,1992.0,Collins,http://images.amazon.com/images/P/0001010565.0...,http://images.amazon.com/images/P/0001010565.0...,http://images.amazon.com/images/P/0001010565.0...,"(Children's fiction, Cats, fiction, Christmas,..."
46713,0001046438,Liar,Stephen Fry,1970.0,Harpercollins Uk,http://images.amazon.com/images/P/0001046438.0...,http://images.amazon.com/images/P/0001046438.0...,http://images.amazon.com/images/P/0001046438.0...,"(Truthfulness and falsehood, Fiction, Murder, ..."
116706,0001046713,Twopence to Cross the Mersey,Helen Forrester,1992.0,HarperCollins Publishers,http://images.amazon.com/images/P/0001046713.0...,http://images.amazon.com/images/P/0001046713.0...,http://images.amazon.com/images/P/0001046713.0...,"(Biographies, Biography, Childhood and youth, ..."
46755,000104687X,"T.S. Eliot Reading \The Wasteland\"" and Other ...",T.S. Eliot,1993.0,HarperCollins Publishers,http://images.amazon.com/images/P/000104687X.0...,http://images.amazon.com/images/P/000104687X.0...,http://images.amazon.com/images/P/000104687X.0...,()
150344,0001046934,The Prime of Miss Jean Brodie,Muriel Spark,1999.0,Trafalgar Square Publishing,http://images.amazon.com/images/P/0001046934.0...,http://images.amazon.com/images/P/0001046934.0...,http://images.amazon.com/images/P/0001046934.0...,"(Teacher-student relationships, Women teachers..."


In [72]:
# Books with duplicates that are ONLY different in 'subjects':
df_test_deduplicated.duplicated(subset=['isbn', 'year_of_publication'], keep=False).sum() # 0
# 6404

0

In [73]:
df_test_deduplicated.head()

,isbn,book_title,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l,subjects
219957,0001010565,Mog's Christmas,Judith Kerr,1992.0,Collins,http://images.amazon.com/images/P/0001010565.0...,http://images.amazon.com/images/P/0001010565.0...,http://images.amazon.com/images/P/0001010565.0...,"(Children's fiction, Cats, fiction, Christmas,..."
46713,0001046438,Liar,Stephen Fry,1970.0,Harpercollins Uk,http://images.amazon.com/images/P/0001046438.0...,http://images.amazon.com/images/P/0001046438.0...,http://images.amazon.com/images/P/0001046438.0...,"(Truthfulness and falsehood, Fiction, Murder, ..."
116706,0001046713,Twopence to Cross the Mersey,Helen Forrester,1992.0,HarperCollins Publishers,http://images.amazon.com/images/P/0001046713.0...,http://images.amazon.com/images/P/0001046713.0...,http://images.amazon.com/images/P/0001046713.0...,"(Biographies, Biography, Childhood and youth, ..."
46755,000104687X,"T.S. Eliot Reading \The Wasteland\"" and Other ...",T.S. Eliot,1993.0,HarperCollins Publishers,http://images.amazon.com/images/P/000104687X.0...,http://images.amazon.com/images/P/000104687X.0...,http://images.amazon.com/images/P/000104687X.0...,()
150344,0001046934,The Prime of Miss Jean Brodie,Muriel Spark,1999.0,Trafalgar Square Publishing,http://images.amazon.com/images/P/0001046934.0...,http://images.amazon.com/images/P/0001046934.0...,http://images.amazon.com/images/P/0001046934.0...,"(Teacher-student relationships, Women teachers..."


In [74]:
df_test_deduplicated.to_csv('data/test_big_cleaned.csv', index=False)

##### Books with duplicates that are ONLY different in 'year_of_publication'

In [75]:
df_test_deduplicated = df_test_deduplicated.sort_values(by='isbn')
df_test_deduplicated = df_test_deduplicated.sort_values(by='year_of_publication')

In [76]:
df_test_deduplicated.isbn.isnull().sum() #0

0

In [77]:
# Books with duplicates that are ONLY different in 'year_of_publication':
df_test_deduplicated.duplicated(subset=['isbn', 'subjects'], keep=False).sum()
# 26

26

In [78]:
df_td2 = df_test_deduplicated.copy()
df_clean = df_td2.dropna(subset=['year_of_publication']) # remove NA values in 'year_of_publication'
df_clean = df_clean.sort_values('year_of_publication', ascending=False)  # Sort to keep the most recent year
df_clean = df_clean.drop_duplicates(subset=['isbn', 'subjects'], keep='first')  # Drop duplicates, keeping the first

In [79]:
print(df_clean.isbn.isnull().sum()) # 0
print(df_clean.duplicated(subset=['isbn', 'subjects'], keep=False).sum()) # 0
print(df_clean.duplicated(subset=['isbn', 'year_of_publication'], keep=False).sum()) # 0
print(df_clean.duplicated().sum()) # 0
print(df_clean.duplicated(subset=['isbn'], keep=False).sum()) # 0
print(df_clean.isbn.isna().sum()) # 0


0
0
0
0
0
0


In [80]:
# Clean up the old indices:
df_clean = df_clean.reset_index(drop=True)
df_clean.head()

,isbn,book_title,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l,subjects
0,0590085417,Heidi,Johanna Spyri,2021.0,Scholastic,http://images.amazon.com/images/P/0590085417.0...,http://images.amazon.com/images/P/0590085417.0...,http://images.amazon.com/images/P/0590085417.0...,"(Johanna Spyri, Shirley Temple, Movie tie-in, ..."
1,068160204X,The Royals,Kitty Kelley,2020.0,Bausch &amp; Lombard,http://images.amazon.com/images/P/068160204X.0...,http://images.amazon.com/images/P/068160204X.0...,http://images.amazon.com/images/P/068160204X.0...,()
2,068107468X,Edgar Allen Poe Collected Poems,Edgar Allan Poe,2020.0,Bausch &amp; Lombard,http://images.amazon.com/images/P/068107468X.0...,http://images.amazon.com/images/P/068107468X.0...,http://images.amazon.com/images/P/068107468X.0...,(American Fantasy poetry)
3,0307124533,Owl's Amazing but True No. 2,Owl Magazine,2012.0,Golden Books,http://images.amazon.com/images/P/0307124533.0...,http://images.amazon.com/images/P/0307124533.0...,http://images.amazon.com/images/P/0307124533.0...,()
4,0380816792,A Rose in Winter,Kathleen E. Woodiwiss,2011.0,Harper Mass Market Paperbacks,http://images.amazon.com/images/P/0380816792.0...,http://images.amazon.com/images/P/0380816792.0...,http://images.amazon.com/images/P/0380816792.0...,"(Fiction, Historical Fiction, Romance, Fiction..."


In [81]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226495 entries, 0 to 226494
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 226495 non-null  object 
 1   book_title           226495 non-null  object 
 2   book_author          226493 non-null  object 
 3   year_of_publication  226495 non-null  float64
 4   publisher            226493 non-null  object 
 5   image_url_s          226495 non-null  object 
 6   image_url_m          226495 non-null  object 
 7   image_url_l          226492 non-null  object 
 8   subjects             226495 non-null  object 
dtypes: float64(1), object(8)
memory usage: 15.6+ MB


In [82]:
df_missing_auth = df_clean[df_clean['book_author'].isna()]
df_missing_auth.head()

,isbn,book_title,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l,subjects
72202,0751352497,A+ Quiz Masters:01 Earth,NaN,1999.0,Dorling Kindersley,http://images.amazon.com/images/P/0751352497.0...,http://images.amazon.com/images/P/0751352497.0...,http://images.amazon.com/images/P/0751352497.0...,()
115299,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN,1995.0,Edinburgh Financial Publishing,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...,()


In [83]:
# Did some googling and found out the author for one of the books:
df_clean.loc[df_clean['isbn'] == '9627982032', 'book_author'] = 'Larissa Downes'

In [84]:
df_clean.loc[df_clean['isbn'] == '9627982032'].head()

,isbn,book_title,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l,subjects
115299,9627982032,The Credit Suisse Guide to Managing Your Perso...,Larissa Downes,1995.0,Edinburgh Financial Publishing,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...,()


In [85]:
df_missing_auth = df_clean[df_clean['book_author'].isna()]
df_missing_auth.head()

,isbn,book_title,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l,subjects
72202,0751352497,A+ Quiz Masters:01 Earth,NaN,1999.0,Dorling Kindersley,http://images.amazon.com/images/P/0751352497.0...,http://images.amazon.com/images/P/0751352497.0...,http://images.amazon.com/images/P/0751352497.0...,()


In [86]:
# # The book 'A+ Quiz Masters:01 Earth' has no information whatsoever 
# (no author, contradicting publisher info and no subjects)
# Let's remove this remaining row:
df_clean = df_clean.dropna(subset=['book_author'])

In [87]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226494 entries, 0 to 226494
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 226494 non-null  object 
 1   book_title           226494 non-null  object 
 2   book_author          226494 non-null  object 
 3   year_of_publication  226494 non-null  float64
 4   publisher            226492 non-null  object 
 5   image_url_s          226494 non-null  object 
 6   image_url_m          226494 non-null  object 
 7   image_url_l          226491 non-null  object 
 8   subjects             226494 non-null  object 
dtypes: float64(1), object(8)
memory usage: 17.3+ MB


In [88]:
df_missing_pub = df_clean[df_clean['publisher'].isna()]
df_missing_pub.head()

,isbn,book_title,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l,subjects
17738,193169656X,Tyrant Moon,Elaine Corvidae,2002.0,NaN,http://images.amazon.com/images/P/193169656X.0...,http://images.amazon.com/images/P/193169656X.0...,http://images.amazon.com/images/P/193169656X.0...,()
41203,1931696993,Finders Keepers,Linnea Sinclair,2001.0,NaN,http://images.amazon.com/images/P/1931696993.0...,http://images.amazon.com/images/P/1931696993.0...,http://images.amazon.com/images/P/1931696993.0...,()


In [89]:
# After some googling, the missings are both from the same publisher, 
# let's complete the missing publisher info:
df_clean.loc[df_clean['isbn'] == '193169656X', 'publisher'] = 'NBI NovelBooks'
df_clean.loc[df_clean['isbn'] == '1931696993', 'publisher'] = 'NBI NovelBooks'

In [90]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226494 entries, 0 to 226494
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 226494 non-null  object 
 1   book_title           226494 non-null  object 
 2   book_author          226494 non-null  object 
 3   year_of_publication  226494 non-null  float64
 4   publisher            226494 non-null  object 
 5   image_url_s          226494 non-null  object 
 6   image_url_m          226494 non-null  object 
 7   image_url_l          226491 non-null  object 
 8   subjects             226494 non-null  object 
dtypes: float64(1), object(8)
memory usage: 17.3+ MB


#### Converting 'subjects' from messy frozenset into Word2Vec embeddings:
Decided to remove this part in the final .csv since it adds huge file size and is easily done later

In [242]:
# !pip install gensim

  Using cached smart_open-7.0.4-py3-none-any.whl.metadata (23 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 4.7 MB/s eta 0:00:00 MB/s eta 0:00:01:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 3.7 MB/s eta 0:00:000:00:010:00:01:01
Using cached smart_open-7.0.4-py3-none-any.whl (61 kB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [244]:
# from gensim.models import KeyedVectors
# from gensim.downloader import load

# # Load pre-trained Word2Vec embeddings
# word_vectors = load('word2vec-google-news-300')  # This is for Word2Vec; use appropriate model

# # Define a function to convert a list of words to an average embedding
# def get_average_embedding(words, model):
#     valid_words = [word for word in words if word in model.key_to_index]
#     if not valid_words:
#         return np.zeros(model.vector_size)
#     embeddings = np.array([model[word] for word in valid_words])
#     return embeddings.mean(axis=0)

# # Assuming df_clean is your DataFrame and 'subjects' column contains frozensets of topics
# def convert_subjects_to_embeddings(df, column_name, model):
#     # Convert frozensets to lists of words
#     df[column_name] = df[column_name].apply(lambda x: list(x))
#     # Apply function to get average embeddings
#     df[column_name + '_embedding'] = df[column_name].apply(lambda x: get_average_embedding(x, model))
#     return df

# # Apply conversion
# df_clean = convert_subjects_to_embeddings(df_clean, 'subjects', word_vectors)

# # Now df_clean contains an additional column with embeddings
# df_clean.head()


[==================================================] 100.0% 1662.8/1662.8MB downloaded


,isbn,book_title,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l,subjects,subjects_embedding
0,0590085417,Heidi,Johanna Spyri,2021.0,Scholastic,http://images.amazon.com/images/P/0590085417.0...,http://images.amazon.com/images/P/0590085417.0...,http://images.amazon.com/images/P/0590085417.0...,"[Johanna Spyri, Shirley Temple, Movie tie-in, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,068160204X,The Royals,Kitty Kelley,2020.0,Bausch &amp; Lombard,http://images.amazon.com/images/P/068160204X.0...,http://images.amazon.com/images/P/068160204X.0...,http://images.amazon.com/images/P/068160204X.0...,[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,068107468X,Edgar Allen Poe Collected Poems,Edgar Allan Poe,2020.0,Bausch &amp; Lombard,http://images.amazon.com/images/P/068107468X.0...,http://images.amazon.com/images/P/068107468X.0...,http://images.amazon.com/images/P/068107468X.0...,[American Fantasy poetry],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0307124533,Owl's Amazing but True No. 2,Owl Magazine,2012.0,Golden Books,http://images.amazon.com/images/P/0307124533.0...,http://images.amazon.com/images/P/0307124533.0...,http://images.amazon.com/images/P/0307124533.0...,[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0380816792,A Rose in Winter,Kathleen E. Woodiwiss,2011.0,Harper Mass Market Paperbacks,http://images.amazon.com/images/P/0380816792.0...,http://images.amazon.com/images/P/0380816792.0...,http://images.amazon.com/images/P/0380816792.0...,"[Fiction, Historical Fiction, Romance, Fiction...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [91]:
# Now turn the frozensets into strings for the model:
df_clean['subjects'] = df_clean['subjects'].apply(lambda x: ', '.join(sorted(x)))
df_clean['subjects'] = df_clean['subjects'].str.replace(r',\s*,+', ', ', regex=True)  # Remove double commas
df_clean['subjects'] = df_clean['subjects'].str.strip()  # Remove leading/trailing spaces
df_clean['subjects'] = df_clean['subjects'].str.replace(' +', ' ', regex=True)  # Replace multiple spaces with a single space


In [92]:
df_clean.head()

,isbn,book_title,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l,subjects
0,0590085417,Heidi,Johanna Spyri,2021.0,Scholastic,http://images.amazon.com/images/P/0590085417.0...,http://images.amazon.com/images/P/0590085417.0...,http://images.amazon.com/images/P/0590085417.0...,"Johanna Spyri, Shirley Temple, Movie tie-in, C..."
1,068160204X,The Royals,Kitty Kelley,2020.0,Bausch &amp; Lombard,http://images.amazon.com/images/P/068160204X.0...,http://images.amazon.com/images/P/068160204X.0...,http://images.amazon.com/images/P/068160204X.0...,
2,068107468X,Edgar Allen Poe Collected Poems,Edgar Allan Poe,2020.0,Bausch &amp; Lombard,http://images.amazon.com/images/P/068107468X.0...,http://images.amazon.com/images/P/068107468X.0...,http://images.amazon.com/images/P/068107468X.0...,American Fantasy poetry
3,0307124533,Owl's Amazing but True No. 2,Owl Magazine,2012.0,Golden Books,http://images.amazon.com/images/P/0307124533.0...,http://images.amazon.com/images/P/0307124533.0...,http://images.amazon.com/images/P/0307124533.0...,
4,0380816792,A Rose in Winter,Kathleen E. Woodiwiss,2011.0,Harper Mass Market Paperbacks,http://images.amazon.com/images/P/0380816792.0...,http://images.amazon.com/images/P/0380816792.0...,http://images.amazon.com/images/P/0380816792.0...,"Fiction, Historical Fiction, Romance, Fiction,..."


In [93]:
# Some final cleaning:
from html import unescape

def clean_string_column(df, column_name):
    # Define a function to clean individual strings
    def clean_string(s):
        if pd.isna(s):  # Check for NaN values
            return s
        s = str(s)  # Ensure the input is a string
        s = unescape(s)  # Replace HTML entities
        s = re.sub(r'\s+', ' ', s)  # Replace multiple spaces with a single space
        s = re.sub(r',+', ',', s)  # Replace multiple commas with a single comma
        s = s.strip()  # Remove leading and trailing spaces
        return s

    # Apply the cleaning function to the specified column
    df[column_name] = df[column_name].apply(clean_string)

clean_string_column(df_clean, 'publisher')
clean_string_column(df_clean, 'book_author')
clean_string_column(df_clean, 'book_title')

In [94]:
# rename column 'subjects' back to 'genre':
df_clean = df_clean.rename(columns={'subjects': 'genre'})

# rename last column to 'genre_embedding':
df_clean = df_clean.rename(columns={'subjects_embedding': 'genre_embedding'})

In [95]:
df_clean.head()

,isbn,book_title,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l,genre
0,0590085417,Heidi,Johanna Spyri,2021.0,Scholastic,http://images.amazon.com/images/P/0590085417.0...,http://images.amazon.com/images/P/0590085417.0...,http://images.amazon.com/images/P/0590085417.0...,"Johanna Spyri, Shirley Temple, Movie tie-in, C..."
1,068160204X,The Royals,Kitty Kelley,2020.0,Bausch & Lombard,http://images.amazon.com/images/P/068160204X.0...,http://images.amazon.com/images/P/068160204X.0...,http://images.amazon.com/images/P/068160204X.0...,
2,068107468X,Edgar Allen Poe Collected Poems,Edgar Allan Poe,2020.0,Bausch & Lombard,http://images.amazon.com/images/P/068107468X.0...,http://images.amazon.com/images/P/068107468X.0...,http://images.amazon.com/images/P/068107468X.0...,American Fantasy poetry
3,0307124533,Owl's Amazing but True No. 2,Owl Magazine,2012.0,Golden Books,http://images.amazon.com/images/P/0307124533.0...,http://images.amazon.com/images/P/0307124533.0...,http://images.amazon.com/images/P/0307124533.0...,
4,0380816792,A Rose in Winter,Kathleen E. Woodiwiss,2011.0,Harper Mass Market Paperbacks,http://images.amazon.com/images/P/0380816792.0...,http://images.amazon.com/images/P/0380816792.0...,http://images.amazon.com/images/P/0380816792.0...,"Fiction, Historical Fiction, Romance, Fiction,..."


In [96]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226494 entries, 0 to 226494
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 226494 non-null  object 
 1   book_title           226494 non-null  object 
 2   book_author          226494 non-null  object 
 3   year_of_publication  226494 non-null  float64
 4   publisher            226494 non-null  object 
 5   image_url_s          226494 non-null  object 
 6   image_url_m          226494 non-null  object 
 7   image_url_l          226491 non-null  object 
 8   genre                226494 non-null  object 
dtypes: float64(1), object(8)
memory usage: 17.3+ MB


In [252]:
# df_clean.to_csv('data/test_big_cleaned.csv', index=False)

In [253]:
# df_clean_no_emb = df_clean.drop(columns='genre_embedding')

In [97]:
df_clean.to_csv('data/df_clean.csv', index=False)

In [98]:
df_clean.head()

,isbn,book_title,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l,genre
0,0590085417,Heidi,Johanna Spyri,2021.0,Scholastic,http://images.amazon.com/images/P/0590085417.0...,http://images.amazon.com/images/P/0590085417.0...,http://images.amazon.com/images/P/0590085417.0...,"Johanna Spyri, Shirley Temple, Movie tie-in, C..."
1,068160204X,The Royals,Kitty Kelley,2020.0,Bausch & Lombard,http://images.amazon.com/images/P/068160204X.0...,http://images.amazon.com/images/P/068160204X.0...,http://images.amazon.com/images/P/068160204X.0...,
2,068107468X,Edgar Allen Poe Collected Poems,Edgar Allan Poe,2020.0,Bausch & Lombard,http://images.amazon.com/images/P/068107468X.0...,http://images.amazon.com/images/P/068107468X.0...,http://images.amazon.com/images/P/068107468X.0...,American Fantasy poetry
3,0307124533,Owl's Amazing but True No. 2,Owl Magazine,2012.0,Golden Books,http://images.amazon.com/images/P/0307124533.0...,http://images.amazon.com/images/P/0307124533.0...,http://images.amazon.com/images/P/0307124533.0...,
4,0380816792,A Rose in Winter,Kathleen E. Woodiwiss,2011.0,Harper Mass Market Paperbacks,http://images.amazon.com/images/P/0380816792.0...,http://images.amazon.com/images/P/0380816792.0...,http://images.amazon.com/images/P/0380816792.0...,"Fiction, Historical Fiction, Romance, Fiction,..."


In [99]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226494 entries, 0 to 226494
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 226494 non-null  object 
 1   book_title           226494 non-null  object 
 2   book_author          226494 non-null  object 
 3   year_of_publication  226494 non-null  float64
 4   publisher            226494 non-null  object 
 5   image_url_s          226494 non-null  object 
 6   image_url_m          226494 non-null  object 
 7   image_url_l          226491 non-null  object 
 8   genre                226494 non-null  object 
dtypes: float64(1), object(8)
memory usage: 17.3+ MB
